In [37]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt
from scipy.stats import beta

In [38]:
##Assume R_EXPR_Bucket3 and R_EXPT_Bucket1 came up as two best buckets 
##with consistent high rewards after 100 iterations

##1.Exploit Only Approach: samples are only picked up from exploit buckets
##2.Explore and Exploit at Random: samples are randomly picked from explore and exploit buckets
##3.Concurrent Arm Beta Sampling: samples are picked from the buckets based on the recent rewards.


In [39]:
df_AL_prog = pd.read_excel("AL_data1.xlsx")

In [40]:
df_AL_prog

,Day,R_EXPR_Bucket1,R_EXPR_Bucket2,R_EXPR_Bucket3,R_EXPR_Bucket4,R_EXPT_Bucket1,R_EXPT_Bucket2,R_EXPT_Bucket3,R_EXPT_Bucket4,SR_EXPR_Bucket1,SR_EXPR_Bucket2,SR_EXPR_Bucket3,SR_EXPR_Bucket4,SR_EXPT_Bucket1,SR_EXPT_Bucket2,SR_EXPT_Bucket3,SR_EXPT_Bucket4
0,1,0.497405,0.031951,0.061860,0.039905,0.148002,0.147913,0.011875,0.033545,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,0.541126,0.081003,0.107044,0.068148,0.120225,0.060056,0.139287,0.072630,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3,0.684797,0.030485,0.076257,0.093349,0.051888,0.037260,0.058451,0.022623,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,4,0.076572,0.059089,0.040310,0.036148,0.053363,0.125806,0.068863,0.024508,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,5,0.223486,0.036944,0.056904,0.049597,0.054044,0.055858,0.090199,0.088001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,6,0.037236,0.028764,0.073113,0.135859,0.029296,0.075040,0.148648,0.077736,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,7,0.253326,0.055542,0.032859,0.010922,0.035297,0.131125,0.126713,0.106387,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,8,0.105901,0.012873,0.034029,0.047224,0.015673,0.057894,0.091646,0.029130,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,9,0.283862,0.081162,0.111623,0.077575,0.015753,0.116151,0.113592,0.068817,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,10,0.566960,0.095745,0.071987,0.091604,0.090816,0.033389,0.000307,0.022791,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [41]:
df_AL_prog.columns

Index(['Day', 'R_EXPR_Bucket1', 'R_EXPR_Bucket2', 'R_EXPR_Bucket3',
       'R_EXPR_Bucket4', 'R_EXPT_Bucket1', 'R_EXPT_Bucket2', 'R_EXPT_Bucket3',
       'R_EXPT_Bucket4', 'SR_EXPR_Bucket1', 'SR_EXPR_Bucket2',
       'SR_EXPR_Bucket3', 'SR_EXPR_Bucket4', 'SR_EXPT_Bucket1',
       'SR_EXPT_Bucket2', 'SR_EXPT_Bucket3', 'SR_EXPT_Bucket4'],
      dtype='object')

In [42]:
###1. Only Exploitation
#####Exploit Only Approach: samples are only picked up from exploit buckets

iter = 0

for iter in range(100):
    
    print(iter)
    if iter == 0 :
        df_AL_prog = pd.read_excel("AL_data1.xlsx")



    max_tot_samples = 15

    curr_day = df_AL_prog[df_AL_prog['R_EXPT_Bucket1'].isnull()].iloc[0,0]


    df_rewards = df_AL_prog[[col for col in df_AL_prog if col.startswith('R')]]


    df_rewards[df_rewards.columns].mean()

    df_SR_cols = [col for col in df_AL_prog if col.startswith('SR')]

    df_team_cols = [col for col in df_AL_prog if col.startswith('R')]


    no_groups = len(df_SR_cols)



    x = np.linspace(0, 1, 1002)[1:-1]
#     x

    ####Get the prior values of a and b based on rewards from last few days and then update 
    ###the sampling ratio for all  buckets by comparing with the previous days sampling ratio  

    for i,j in zip(df_team_cols,df_SR_cols):
        #print(i)
        #print(j)
        if (i.startswith('R_EXPT')):
            a = 8 + random.random()
            b = random.random()
        else:
            a = random.random()
            b = 8 + random.random()
            

        ###Get a random sample for sampling ratio from the beta distribution for the current team 
        ###based on last 3 weeks rewards
        random_beta = np.random.beta(a, b)

        ###Get the last week's sampling ratio for the team 
        max_beta = (df_AL_prog[~df_AL_prog[i].isnull()].tail(3)[i].mean() ) / max_tot_samples


        df_AL_prog[j].iloc[curr_day - 1] = random_beta

    sum_SR = 0
    for col in df_SR_cols:

        sum_SR = sum_SR + df_AL_prog[col].iloc[curr_day - 1]

    for col in df_SR_cols:
        if sum_SR  != 0:
            df_AL_prog[col].iloc[curr_day - 1] = math.floor( (df_AL_prog[col].iloc[curr_day - 1] / sum_SR)* max_tot_samples)



    ####Randomly generate the rewards for next day
    for i in df_team_cols:
        if i == 'R_EXPT_Bucket1':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        elif  i == 'R_EXPR_Bucket3':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        else:
            df_AL_prog[i].iloc[curr_day - 1] = (0 + (random.random()* 0.4 ) )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [43]:
df_AL_prog.head(100)

,Day,R_EXPR_Bucket1,R_EXPR_Bucket2,R_EXPR_Bucket3,R_EXPR_Bucket4,R_EXPT_Bucket1,R_EXPT_Bucket2,R_EXPT_Bucket3,R_EXPT_Bucket4,SR_EXPR_Bucket1,SR_EXPR_Bucket2,SR_EXPR_Bucket3,SR_EXPR_Bucket4,SR_EXPT_Bucket1,SR_EXPT_Bucket2,SR_EXPT_Bucket3,SR_EXPT_Bucket4
0,1,0.497405,0.031951,0.061860,0.039905,0.148002,0.147913,0.011875,0.033545,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,0.541126,0.081003,0.107044,0.068148,0.120225,0.060056,0.139287,0.072630,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3,0.684797,0.030485,0.076257,0.093349,0.051888,0.037260,0.058451,0.022623,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,4,0.076572,0.059089,0.040310,0.036148,0.053363,0.125806,0.068863,0.024508,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,5,0.223486,0.036944,0.056904,0.049597,0.054044,0.055858,0.090199,0.088001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,6,0.037236,0.028764,0.073113,0.135859,0.029296,0.075040,0.148648,0.077736,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,7,0.253326,0.055542,0.032859,0.010922,0.035297,0.131125,0.126713,0.106387,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,8,0.105901,0.012873,0.034029,0.047224,0.015673,0.057894,0.091646,0.029130,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,9,0.283862,0.081162,0.111623,0.077575,0.015753,0.116151,0.113592,0.068817,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,10,0.566960,0.095745,0.071987,0.091604,0.090816,0.033389,0.000307,0.022791,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [44]:
###2. Explore and Exploit at Random
###If sampling is done at random

iter = 0

for iter in range(100):
    
    print(iter)
    if iter == 0 :
        df_AL_prog = pd.read_excel("AL_data1.xlsx")



    max_tot_samples = 15

    curr_day = df_AL_prog[df_AL_prog['R_EXPT_Bucket1'].isnull()].iloc[0,0]


    df_rewards = df_AL_prog[[col for col in df_AL_prog if col.startswith('R')]]

    ####Incremental Benefit till now
    df_rewards[df_rewards.columns].mean()

    df_SR_cols = [col for col in df_AL_prog if col.startswith('SR')]

    df_team_cols = [col for col in df_AL_prog if col.startswith('R')]


    no_groups = len(df_SR_cols)

    ##Generate Sample data for tracking Beta distribution for different buckets
    x = np.linspace(0, 1, 1002)[1:-1]


    ####Get the prior values of a and b based on rewards from last few days and then update 
    ###the sampling ratio for all  teams by comparing with the previous days sampling ratio  

    for i,j in zip(df_team_cols,df_SR_cols):

        a = random.random()*10
        b = random.random()*10

        ###Get a random sample for sampling ratio from the beta distribution for the current team 
        ###based on last 3 weeks rewards
        random_beta = np.random.beta(a, b)

        ###Get the last week's sampling ratio for the team 
        max_beta = (df_AL_prog[~df_AL_prog[i].isnull()].tail(3)[i].mean() ) / max_tot_samples


        ###Updating the sampling ratios of the buckets for the next day
  
        df_AL_prog[j].iloc[curr_day - 1] = random_beta

    sum_SR = 0
    for col in df_SR_cols:

        sum_SR = sum_SR + df_AL_prog[col].iloc[curr_day - 1]

    for col in df_SR_cols:
        if sum_SR  != 0:
            df_AL_prog[col].iloc[curr_day - 1] = math.floor( (df_AL_prog[col].iloc[curr_day - 1] / sum_SR)* max_tot_samples)



    ####Randomly generate the rewards for next week. In the application, we will get from the user feedback table
    for i in df_team_cols:
        if i == 'R_EXPT_Bucket1':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        elif  i == 'R_EXPR_Bucket3':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        else:
            df_AL_prog[i].iloc[curr_day - 1] = (0 + (random.random()* 0.4 ) )

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [45]:
df_AL_prog.head(100)

,Day,R_EXPR_Bucket1,R_EXPR_Bucket2,R_EXPR_Bucket3,R_EXPR_Bucket4,R_EXPT_Bucket1,R_EXPT_Bucket2,R_EXPT_Bucket3,R_EXPT_Bucket4,SR_EXPR_Bucket1,SR_EXPR_Bucket2,SR_EXPR_Bucket3,SR_EXPR_Bucket4,SR_EXPT_Bucket1,SR_EXPT_Bucket2,SR_EXPT_Bucket3,SR_EXPT_Bucket4
0,1,0.497405,0.031951,0.061860,0.039905,0.148002,0.147913,0.011875,0.033545,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,0.541126,0.081003,0.107044,0.068148,0.120225,0.060056,0.139287,0.072630,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3,0.684797,0.030485,0.076257,0.093349,0.051888,0.037260,0.058451,0.022623,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,4,0.076572,0.059089,0.040310,0.036148,0.053363,0.125806,0.068863,0.024508,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,5,0.223486,0.036944,0.056904,0.049597,0.054044,0.055858,0.090199,0.088001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,6,0.037236,0.028764,0.073113,0.135859,0.029296,0.075040,0.148648,0.077736,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,7,0.253326,0.055542,0.032859,0.010922,0.035297,0.131125,0.126713,0.106387,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,8,0.105901,0.012873,0.034029,0.047224,0.015673,0.057894,0.091646,0.029130,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,9,0.283862,0.081162,0.111623,0.077575,0.015753,0.116151,0.113592,0.068817,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,10,0.566960,0.095745,0.071987,0.091604,0.090816,0.033389,0.000307,0.022791,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [46]:
###3. Concurrent Arm Beta Sampling
iter = 0

for iter in range(100):
    
    print(iter)
    if iter == 0 :
        df_AL_prog = pd.read_excel("AL_data1.xlsx")



    max_tot_samples = 15

    curr_day = df_AL_prog[df_AL_prog['R_EXPT_Bucket1'].isnull()].iloc[0,0]


    df_rewards = df_AL_prog[[col for col in df_AL_prog if col.startswith('R')]]

    ####Incremental Benefit till now
    df_rewards[df_rewards.columns].mean()

    df_SR_cols = [col for col in df_AL_prog if col.startswith('SR')]

    df_team_cols = [col for col in df_AL_prog if col.startswith('R')]



    no_groups = len(df_SR_cols)


    ##Generate Sample data for tracking Beta distribution for different buckets
    x = np.linspace(0, 1, 1002)[1:-1]


    ####Get the prior values of a and b based on rewards from last few days and then update 
    ###the sampling ratio for all  buckets by comparing with the previous day's sampling ratio  
    ###If sampling ratio < 
    for i,j in zip(df_team_cols,df_SR_cols):
 
        a = df_AL_prog[i].mean() * 10
        b = (1 - df_AL_prog[i].mean()) * 10 
        print('a = '+ str(a))
        print('b = '+ str(b))
        print('\n')

#         ####Visulaize the beta distribution for the current team based on last 3 weeks rewards
#         dist = beta(a, b)
#         plt.plot(x, dist.pdf(x), c='black',label=r'$\alpha=%.1f,\ \beta=%.1f$' % (a, b))
#         plt.xlim(0, 1)
#         plt.ylim(0, 3)
#         plt.xlabel('$x$')
#         plt.ylabel(r'$p(x|\alpha,\beta)$')
#         plt.title('Beta Distribution')
#         plt.legend(loc=0)
#         plt.show()

        ###Get a random sample for sampling ratio from the beta distribution for the current bucket 
        ###based on last 3 days rewards
        random_beta = np.random.beta(a, b)

        ###Get the last week's sampling ratio for the team 
        max_beta = (df_AL_prog[~df_AL_prog[i].isnull()].tail(3)[i].mean() ) / max_tot_samples
#         print('max_sr '+ str(max_beta))
#         if random_beta > max_beta:
# #             print('Beta Updated')
#             max_beta = random_beta #update the maximum beta value thus far observed
#             print('max_sr_updated '+ str(max_beta) )
#             print('\n')
        max_beta = random_beta

        ###Updating the sampling ratios of the teams for the next day
        df_AL_prog[j].iloc[curr_day - 1] = max_beta

    sum_SR = 0
    for col in df_SR_cols:

        sum_SR = sum_SR + df_AL_prog[col].iloc[curr_day - 1]
#         print('sum_sr  ' + str(sum_SR) )
    for col in df_SR_cols:
        if sum_SR  != 0:
            df_AL_prog[col].iloc[curr_day - 1] = math.ceil( (df_AL_prog[col].iloc[curr_day - 1] / sum_SR)* max_tot_samples)



    ####Randomly generate the rewards for next day
    for i in df_team_cols:
        if i == 'R_EXPT_Bucket1':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        elif  i == 'R_EXPR_Bucket3':
            df_AL_prog[i].iloc[curr_day - 1] = (0.7 + (random.random()* 0.2 ) )
        else:
            df_AL_prog[i].iloc[curr_day - 1] = (0 + (random.random()* 0.4 ) )

0
a = 3.2706699915640964
b = 6.729330008435904


a = 0.5135589400904085
b = 9.486441059909591


a = 0.6659862547519089
b = 9.334013745248091


a = 0.6503319750864001
b = 9.3496680249136


a = 0.6143561738608757
b = 9.385643826139123


a = 0.8404928529085955
b = 9.159507147091405


a = 0.8495810161519284
b = 9.150418983848072


a = 0.5461683329957323
b = 9.453831667004268


1
a = 3.1186546002610744
b = 6.881345399738925


a = 0.7384430880187721
b = 9.261556911981227


a = 1.341772746329382
b = 8.658227253670619


a = 0.6279501614165716
b = 9.372049838583429


a = 1.361672025224774
b = 8.638327974775226


a = 0.9876815544416377
b = 9.012318445558362


a = 1.065678569035715
b = 8.934321430964285


a = 0.6427210199495293
b = 9.35727898005047


2
a = 2.879039957026295
b = 7.120960042973705


a = 0.6889922474747835
b = 9.311007752525217


a = 1.9709632978485767
b = 8.029036702151423


a = 0.7814929906382402
b = 9.21850700936176


a = 1.9047837603422988
b = 8.0952162396577


a = 1.16635414367

22
a = 2.130741577861656
b = 7.869258422138344


a = 1.6654702551402774
b = 8.334529744859722


a = 5.745022756873345
b = 4.254977243126655


a = 1.597244441307625
b = 8.402755558692375


a = 5.745198772544853
b = 4.254801227455147


a = 1.4242930074924796
b = 8.57570699250752


a = 1.7829432999839199
b = 8.21705670001608


a = 1.4174233106221212
b = 8.582576689377879


23
a = 2.1361422830218166
b = 7.863857716978183


a = 1.721610585702729
b = 8.278389414297271


a = 5.785193089012118
b = 4.214806910987882


a = 1.64763672190119
b = 8.35236327809881


a = 5.808273969888949
b = 4.191726030111051


a = 1.4385755279463621
b = 8.561424472053638


a = 1.826907490915688
b = 8.173092509084313


a = 1.375409137718107
b = 8.624590862281893


24
a = 2.0851477964969094
b = 7.9148522035030915


a = 1.71445740823748
b = 8.285542591762521


a = 5.874272654514321
b = 4.125727345485679


a = 1.6175910364763888
b = 8.382408963523611


a = 5.863276487068209
b = 4.136723512931791


a = 1.452277411775772

44
a = 2.1407999625579706
b = 7.859200037442029


a = 1.8828878574105412
b = 8.117112142589459


a = 6.774666155528814
b = 3.2253338444711863


a = 1.7609334968489367
b = 8.239066503151063


a = 6.6717366837216066
b = 3.3282633162783934


a = 1.6578388318384418
b = 8.342161168161558


a = 1.8273606901064985
b = 8.172639309893501


a = 1.5781850913484405
b = 8.421814908651559


45
a = 2.148754860096995
b = 7.851245139903005


a = 1.8565702967360331
b = 8.143429703263967


a = 6.810528429242276
b = 3.189471570757724


a = 1.787781203106875
b = 8.212218796893126


a = 6.681637465758746
b = 3.318362534241255


a = 1.6335926210484422
b = 8.366407378951559


a = 1.8242212768618715
b = 8.175778723138128


a = 1.5975461486160947
b = 8.402453851383905


46
a = 2.171860903409619
b = 7.82813909659038


a = 1.8600118918492057
b = 8.139988108150794


a = 6.843205373501085
b = 3.1567946264989146


a = 1.8157656605446595
b = 8.18423433945534


a = 6.7171157830817325
b = 3.282884216918267


a = 1.6581

a = 1.7132894098153908
b = 8.28671059018461


66
a = 2.172726601805313
b = 7.827273398194688


a = 1.963455199590947
b = 8.036544800409054


a = 7.136813454271249
b = 2.863186545728751


a = 1.8629852238112838
b = 8.137014776188716


a = 7.042868841842525
b = 2.9571311581574755


a = 1.7737578524197264
b = 8.226242147580274


a = 1.7552527961755768
b = 8.244747203824422


a = 1.71415256403751
b = 8.28584743596249


67
a = 2.1899417086686785
b = 7.8100582913313215


a = 1.9423703115323816
b = 8.057629688467618


a = 7.136404297642015
b = 2.863595702357985


a = 1.8641285013346252
b = 8.135871498665374


a = 7.049113172567546
b = 2.950886827432454


a = 1.7637870006943257
b = 8.236212999305675


a = 1.7762881968623436
b = 8.223711803137657


a = 1.743406372158396
b = 8.256593627841603


68
a = 2.212920737711624
b = 7.787079262288376


a = 1.9533883713340563
b = 8.046611628665945


a = 7.159405469940201
b = 2.840594530059799


a = 1.8431439704224037
b = 8.156856029577597


a = 7.061997909

88
a = 2.181867471084422
b = 7.818132528915578


a = 1.8443978691070382
b = 8.155602130892962


a = 7.311075129564797
b = 2.688924870435203


a = 1.9307652966069853
b = 8.069234703393015


a = 7.226447945278522
b = 2.7735520547214776


a = 1.816518056834058
b = 8.183481943165942


a = 1.702596838190467
b = 8.297403161809534


a = 1.7515104031727127
b = 8.248489596827287


89
a = 2.1799576825777054
b = 7.820042317422295


a = 1.8428421202420218
b = 8.157157879757978


a = 7.312018761784942
b = 2.687981238215058


a = 1.925794503151307
b = 8.074205496848693


a = 7.232888811202169
b = 2.7671111887978306


a = 1.8369023655018824
b = 8.163097634498119


a = 1.718128749948398
b = 8.281871250051601


a = 1.740495199620066
b = 8.259504800379935


90
a = 2.175727134885395
b = 7.824272865114605


a = 1.8370064149904228
b = 8.162993585009577


a = 7.3152760106936405
b = 2.684723989306359


a = 1.9301520997092545
b = 8.069847900290746


a = 7.236367220653729
b = 2.7636327793462714


a = 1.8456619

In [47]:
df_AL_prog.head(100)

,Day,R_EXPR_Bucket1,R_EXPR_Bucket2,R_EXPR_Bucket3,R_EXPR_Bucket4,R_EXPT_Bucket1,R_EXPT_Bucket2,R_EXPT_Bucket3,R_EXPT_Bucket4,SR_EXPR_Bucket1,SR_EXPR_Bucket2,SR_EXPR_Bucket3,SR_EXPR_Bucket4,SR_EXPT_Bucket1,SR_EXPT_Bucket2,SR_EXPT_Bucket3,SR_EXPT_Bucket4
0,1,0.497405,0.031951,0.061860,0.039905,0.148002,0.147913,0.011875,0.033545,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,2,0.541126,0.081003,0.107044,0.068148,0.120225,0.060056,0.139287,0.072630,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
2,3,0.684797,0.030485,0.076257,0.093349,0.051888,0.037260,0.058451,0.022623,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,4,0.076572,0.059089,0.040310,0.036148,0.053363,0.125806,0.068863,0.024508,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,5,0.223486,0.036944,0.056904,0.049597,0.054044,0.055858,0.090199,0.088001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
5,6,0.037236,0.028764,0.073113,0.135859,0.029296,0.075040,0.148648,0.077736,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
6,7,0.253326,0.055542,0.032859,0.010922,0.035297,0.131125,0.126713,0.106387,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
7,8,0.105901,0.012873,0.034029,0.047224,0.015673,0.057894,0.091646,0.029130,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
8,9,0.283862,0.081162,0.111623,0.077575,0.015753,0.116151,0.113592,0.068817,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
9,10,0.566960,0.095745,0.071987,0.091604,0.090816,0.033389,0.000307,0.022791,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
